## INKERS EIP : End-to-End Deep Learning Project 
| Intern Name: Niranjan Anandakumar          |  Batch A | nirunitk@gmail.com | +91-9833499066 | 

LinkedIn : https://www.linkedin.com/in/niranjanaryan/
### Project Description
A Blog on Implementation of Research Paper by Yuanfudao and Team Released in March 2018 , latest Release in May  2018

## SemEval-2018 Task 11: Three-way Attention and Relational Knowledge for Commonsense Machine Comprehension

Authors : Liang Wang, Meng Sun, Wei Zhao, Kewei Shen, Jingming Liu, from Yuanfudao Research Beijing China

email { wangliang01,sunmeng,zhaowei01,shenkw,liujm } @fenbi.com

#### Model Overview
We use Attention-Based LSTM Networks.
For more technical details, please refer to our paper at https://arxiv.org/abs/1803.00191

For more details about this task, please refer to paper SemEval-2018 Task 11: Machine Comprehension Using Commonsense Knowledge.


## Packages Prerequisite

python 3.5

pytorch >= 0.2

spacy >= 2.0

#### GPU machine is preferred, training on CPU will be much slower.

### 1. Data Collection and Preparation

Download preprocessed data from [Google Drive](https://drive.google.com/open?id=1M1saVYk-4Xh0Y0Ok6e8liDLnElnGc0P4) or [Baidu Cloud Disk](https://pan.baidu.com/s/1kWHj2z9), unzip and put them under folder data/.

If you choose to preprocess dataset by yourself,
please preprocess official dataset by `python3 preprocess.py`, download [Glove embeddings](http://nlp.stanford.edu/data/glove.840B.300d.zip),
and also remember to download [ConceptNet](https://github.com/commonsense/conceptnet5/wiki/Downloads) and preprocess it with 
`python3 preprocess.py conceptnet`

Official dataset can be downloaded on [hidrive](https://my.hidrive.com/lnk/DhAhE8B5).

We transform original XML format data to Json format with [xml2json](https://github.com/hay/xml2json) by running `./xml2json.py --pretty --strip_text -t xml2json -o test-data.json test-data.xml`


### 2. Building an Architechture 

The overall model architecture is shown below:

![Three-way Attentive Networks](TriAN.jpg)

In this paper, we present Three-way Attentive Networks(TriAN) for multiple-choice commonsense reading comprehension.

The given task requires modeling interactions between the passage, question and answers. Different questions need to
focus on different parts of the passage, attention mechanism is a natural choice and turns out to be effective for 
reading comprehension.

Due to the relatively  small  size  of  training  data, TriAN use word-level attention and 
consists of only one layer of LSTM(Hochreiter and Schmidhuber, 1997). Deeper models result in serious overfitting and
poor generalization empirically. To explicitly model commonsense knowledge, relation embeddings based on ConceptNet(Speeret al.,2017)
are   used   as   additional   features.


ConceptNet is a large-scale graph of general knowledge from both crowdsourced resources and expert-created resources. 
It  consists  of  over 21 million edges and 8 million nodes. ConceptNet shows state-of-the-art performance on tasks like
word analogy and word relatedness.

Besides, We also find that pretraining our network on other datasets helps to improve the overall performance. There  are  some  existing  multiple-choice English reading comprehension datasets contributed by NLP community such as MCTest(Richardson et al., 2013) and RACE(Lai et al.,2017). 

Although those datasets don’t focus specifically on commonsense comprehension, they provide a convenient way for data augmentation.
Augmented data can be used to learn shared regularities of reading comprehension tasks. Combining all of the aforementioned techniques,
our system  achieves  competitive  performance on the official test set

In [13]:
import os
import sys; sys.argv=['']; del sys
from __future__ import division

### We build a Configuration for the Model and most important Parameters

In [14]:
# Configuration
import os
import argparse
import logging

logger = logging.getLogger(__name__)

def str2bool(v):
    return v.lower() in ('yes', 'true', 't', '1', 'y')

parser = argparse.ArgumentParser()
parser.register('type', 'bool', str2bool)
parser.add_argument('--gpu', type=str, default='0', help='GPU to use')
parser.add_argument('--epoch', type=int, default=1, help='Number of epoches to run')
parser.add_argument('--optimizer', type=str, default='adamax', help='optimizer, adamax or sgd')
parser.add_argument('--use_cuda', type='bool', default=True, help='use cuda or not')
parser.add_argument('--grad_clipping', type=float, default=10.0, help='maximum L2 norm for gradient clipping')
parser.add_argument('--lr', type=float, default=2e-3, help='learning rate')
parser.add_argument('--batch_size', type=int, default=32, help='batch size')
parser.add_argument('--embedding_file', type=str, default='./data/glove.840B.300d.txt', help='embedding file')
parser.add_argument('--hidden_size', type=int, default=96, help='default size for RNN layer')
parser.add_argument('--doc_layers', type=int, default=1, help='number of RNN layers for doc encoding')
parser.add_argument('--rnn_type', type=str, default='lstm', help='RNN type, lstm or gru')
parser.add_argument('--dropout_rnn_output', type=float, default=0.4, help='dropout for RNN output')
parser.add_argument('--rnn_padding', type='bool', default=True, help='Use padding or not')
parser.add_argument('--dropout_emb', type=float, default=0.4, help='dropout rate for embeddings')
parser.add_argument('--pretrained', type=str, default='', help='pretrained model path')
parser.add_argument('--finetune_topk', type=int, default=10, help='Finetune topk embeddings during training')
parser.add_argument('--pos_emb_dim', type=int, default=12, help='Embedding dimension for part-of-speech')
parser.add_argument('--ner_emb_dim', type=int, default=8, help='Embedding dimension for named entities')
parser.add_argument('--rel_emb_dim', type=int, default=10, help='Embedding dimension for ConceptNet relations')
parser.add_argument('--seed', type=int, default=1234, help='random seed')
parser.add_argument('--test_mode', type='bool', default=False, help='In test mode, validation data will be used for training')
args = parser.parse_args()

print(args)

if args.pretrained:
    assert all(os.path.exists(p) for p in args.pretrained.split(',')), 'Checkpoint %s does not exist.' % args.pretrained


Namespace(batch_size=32, doc_layers=1, dropout_emb=0.4, dropout_rnn_output=0.4, embedding_file='./data/glove.840B.300d.txt', epoch=1, finetune_topk=10, gpu='0', grad_clipping=10.0, hidden_size=96, lr=0.002, ner_emb_dim=8, optimizer='adamax', pos_emb_dim=12, pretrained='', rel_emb_dim=10, rnn_padding=True, rnn_type='lstm', seed=1234, test_mode=False, use_cuda=True)


In [15]:
# the Utills for the Pytorch model to Be Defined 
# NLP for Tokensize and  Prepare the bog of word for Model

In [16]:
from conceptnet import * 

Load 657367 triples from ./data/concept.filter


In [17]:
import os
import json
import string
import wikiwords
import unicodedata
import numpy as np

from collections import Counter
from nltk.corpus import stopwords

words = frozenset(stopwords.words('english'))
punc = frozenset(string.punctuation)
def is_stopword(w):
    return w.lower() in words

def is_punc(c):
    return c in punc

baseline = wikiwords.freq('the')
def get_idf(w):
    return np.log(baseline / (wikiwords.freq(w.lower()) + 1e-10))

def load_data(path):
    from doc import Example
    data = []
    for line in open(path, 'r', encoding='utf-8'):
        if path.find('race') < 0 or np.random.random() < 0.6:
            data.append(Example(json.loads(line)))
    print('Load %d examples from %s...' % (len(data), path))
    return data

class Dictionary(object):
    NULL = '<NULL>'
    UNK = '<UNK>'
    START = 2

    @staticmethod
    def normalize(token):
        return unicodedata.normalize('NFD', token)

    def __init__(self):
        self.tok2ind = {self.NULL: 0, self.UNK: 1}
        self.ind2tok = {0: self.NULL, 1: self.UNK}

    def __len__(self):
        return len(self.tok2ind)

    def __iter__(self):
        return iter(self.tok2ind)

    def __contains__(self, key):
        if type(key) == int:
            return key in self.ind2tok
        elif type(key) == str:
            return self.normalize(key) in self.tok2ind

    def __getitem__(self, key):
        if type(key) == int:
            return self.ind2tok.get(key, self.UNK)
        if type(key) == str:
            return self.tok2ind.get(self.normalize(key),
                                    self.tok2ind.get(self.UNK))

    def __setitem__(self, key, item):
        if type(key) == int and type(item) == str:
            self.ind2tok[key] = item
        elif type(key) == str and type(item) == int:
            self.tok2ind[key] = item
        else:
            raise RuntimeError('Invalid (key, item) types.')

    def add(self, token):
        token = self.normalize(token)
        if token not in self.tok2ind:
            index = len(self.tok2ind)
            self.tok2ind[token] = index
            self.ind2tok[index] = token

    def tokens(self):
        """Get dictionary tokens.

        Return all the words indexed by this dictionary, except for special
        tokens.
        """
        tokens = [k for k in self.tok2ind.keys()
                  if k not in {'<NULL>', '<UNK>'}]
        return tokens

vocab, pos_vocab, ner_vocab, rel_vocab = Dictionary(), Dictionary(), Dictionary(), Dictionary()
def gen_race_vocab(data):
    race_vocab = Dictionary()
    build_vocab()
    cnt = Counter()
    for ex in data:
        cnt += Counter(ex.passage.split())
        cnt += Counter(ex.question.split())
        cnt += Counter(ex.choice.split())
    for key, val in cnt.most_common(30000):
        if key not in vocab:
            race_vocab.add(key)
    print('Vocabulary size: %d' % len(race_vocab))
    writer = open('./data/race_vocab', 'w', encoding='utf-8')
    writer.write('\n'.join(race_vocab.tokens()))
    writer.close()

def build_vocab(data=None):
    global vocab, pos_vocab, ner_vocab, rel_vocab
    # build word vocabulary
    if os.path.exists('./data/vocab'):
        print('Load vocabulary from ./data/vocab...')
        for w in open('./data/vocab', encoding='utf-8'):
            vocab.add(w.strip())
        print('Vocabulary size: %d' % len(vocab))
    else:
        cnt = Counter()
        for ex in data:
            cnt += Counter(ex.passage.split())
            cnt += Counter(ex.question.split())
            cnt += Counter(ex.choice.split())
        for key, val in cnt.most_common():
            vocab.add(key)
        print('Vocabulary size: %d' % len(vocab))
        writer = open('./data/vocab', 'w', encoding='utf-8')
        writer.write('\n'.join(vocab.tokens()))
        writer.close()
    # build part-of-speech vocabulary
    if os.path.exists('./data/pos_vocab'):
        print('Load pos vocabulary from ./data/pos_vocab...')
        for w in open('./data/pos_vocab', encoding='utf-8'):
            pos_vocab.add(w.strip())
        print('POS vocabulary size: %d' % len(pos_vocab))
    else:
        cnt = Counter()
        for ex in data:
            cnt += Counter(ex.d_pos)
            cnt += Counter(ex.q_pos)
        for key, val in cnt.most_common():
            if key: pos_vocab.add(key)
        print('POS vocabulary size: %d' % len(pos_vocab))
        writer = open('./data/pos_vocab', 'w', encoding='utf-8')
        writer.write('\n'.join(pos_vocab.tokens()))
        writer.close()
    # build named entity vocabulary
    if os.path.exists('./data/ner_vocab'):
        print('Load ner vocabulary from ./data/ner_vocab...')
        for w in open('./data/ner_vocab', encoding='utf-8'):
            ner_vocab.add(w.strip())
        print('NER vocabulary size: %d' % len(ner_vocab))
    else:
        cnt = Counter()
        for ex in data:
            cnt += Counter(ex.d_ner)
        for key, val in cnt.most_common():
            if key: ner_vocab.add(key)
        print('NER vocabulary size: %d' % len(ner_vocab))
        writer = open('./data/ner_vocab', 'w', encoding='utf-8')
        writer.write('\n'.join(ner_vocab.tokens()))
        writer.close()
    # Load conceptnet relation vocabulary
    assert os.path.exists('./data/rel_vocab')
    print('Load relation vocabulary from ./data/rel_vocab...')
    for w in open('./data/rel_vocab', encoding='utf-8'):
        rel_vocab.add(w.strip())
    print('Rel vocabulary size: %d' % len(rel_vocab))

def gen_submission(data, prediction):
    assert len(data) == len(prediction)
    writer = open('out-%d.txt' % np.random.randint(10**18), 'w', encoding='utf-8')
    for p, ex in zip(prediction, data):
        p_id, q_id, c_id = ex.id.split('_')[-3:]
        writer.write('%s,%s,%s,%f\n' % (p_id, q_id, c_id, p))
    writer.close()

def gen_debug_file(data, prediction):
    writer = open('./data/output.log', 'w', encoding='utf-8')
    cur_pred, cur_choices = [], []
    for i, ex in enumerate(data):
        if i + 1 == len(data):
            cur_pred.append(prediction[i])
            cur_choices.append(ex.choice)
        if (i > 0 and ex.id[:-1] != data[i - 1].id[:-1]) or (i + 1 == len(data)):
            writer.write('Passage: %s\n' % data[i - 1].passage)
            writer.write('Question: %s\n' % data[i - 1].question)
            for idx, choice in enumerate(cur_choices):
                writer.write('%s  %f\n' % (choice, cur_pred[idx]))
            writer.write('\n')
            cur_pred, cur_choices = [], []
        cur_pred.append(prediction[i])
        cur_choices.append(ex.choice)

    writer.close()

def gen_final_submission(data):
    import glob
    proba_list = []
    for f in glob.glob('./out-*.txt'):
        print('Process %s...' % f)
        lines = open(f, 'r', encoding='utf-8').readlines()
        lines = map(lambda s: s.strip(), lines)
        lines = list(filter(lambda s: len(s) > 0, lines))
        assert len(lines) == len(data)
        proba_list.append(lines)
    avg_proba, p_q_id = [], []
    for i in range(len(data)):
        cur_avg_p = np.average([float(p[i].split(',')[-1]) for p in proba_list])
        cur_p_q_id = ','.join(data[i].id.split('_')[-3:-1])
        if i == 0 or cur_p_q_id != p_q_id[-1]:
            avg_proba.append([cur_avg_p])
            p_q_id.append(cur_p_q_id)
        else:
            avg_proba[-1].append(cur_avg_p)
    gen_debug_file(data, [p for sublist in avg_proba for p in sublist])
    writer = open('answer.txt', 'w', encoding='utf-8')
    assert len(avg_proba) == len(p_q_id)
    cnt = 0
    for probas, cur_p_q_id in zip(avg_proba, p_q_id):
        cnt += 1
        assert len(probas) > 1
        pred_ans = np.argmax(probas)
        writer.write('%s,%d' % (cur_p_q_id, pred_ans))
        if cnt < len(p_q_id):
            writer.write('\n')
    writer.close()
    os.system('zip final_output.zip answer.txt')
    print('Please submit final_output.zip to codalab.')

def eval_based_on_outputs(path):
    dev_data = load_data('./data/dev-data-processed.json')
    label = [int(ex.label) for ex in dev_data]
    gold, cur_gold = [], []
    for i, ex in enumerate(dev_data):
        if i + 1 == len(dev_data):
            cur_gold.append(label[i])
        if (i > 0 and ex.id[:-1] != dev_data[i - 1].id[:-1]) or (i + 1 == len(dev_data)):
            gy = np.argmax(cur_gold)
            gold.append(gy)
            cur_gold = []
        cur_gold.append(label[i])
    prediction = [s.strip() for s in open(path, 'r', encoding='utf-8').readlines() if len(s.strip()) > 0]
    prediction = [int(s.split(',')[-1]) for s in prediction]
    assert len(prediction) == len(gold)
    acc = sum([int(p == g) for p, g in zip(prediction, gold)]) / len(gold)
    print('Accuracy on dev_data: %f' % acc)

if __name__ == '__main__':
    # build_vocab()
    trial_data = load_data('./data/trial-data-processed.json')
    train_data = load_data('./data/train-data-processed.json')
    dev_data = load_data('./data/dev-data-processed.json')
    test_data = load_data('./data/test-data-processed.json')
    build_vocab(trial_data + train_data + dev_data + test_data)


Load 1020 examples from ./data/trial-data-processed.json...
Load 19462 examples from ./data/train-data-processed.json...
Load 2822 examples from ./data/dev-data-processed.json...
Load 5594 examples from ./data/test-data-processed.json...
Load vocabulary from ./data/vocab...
Vocabulary size: 33382
Load pos vocabulary from ./data/pos_vocab...
POS vocabulary size: 51
Load ner vocabulary from ./data/ner_vocab...
NER vocabulary size: 20
Load relation vocabulary from ./data/rel_vocab...
Rel vocabulary size: 39


In [18]:
import torch
import numpy as np

from utils import vocab, pos_vocab, ner_vocab, rel_vocab

class Example:

    def __init__(self, input_dict):
        self.id = input_dict['id']
        self.passage = input_dict['d_words']
        self.question = input_dict['q_words']
        self.choice = input_dict['c_words']
        self.d_pos = input_dict['d_pos']
        self.d_ner = input_dict['d_ner']
        self.q_pos = input_dict['q_pos']
        assert len(self.q_pos) == len(self.question.split()), (self.q_pos, self.question)
        assert len(self.d_pos) == len(self.passage.split())
        self.features = np.stack([input_dict['in_q'], input_dict['in_c'], \
                                    input_dict['lemma_in_q'], input_dict['lemma_in_c'], \
                                    input_dict['tf']], 1)
        assert len(self.features) == len(self.passage.split())
        self.label = input_dict['label']

        self.d_tensor = torch.LongTensor([vocab[w] for w in self.passage.split()])
        self.q_tensor = torch.LongTensor([vocab[w] for w in self.question.split()])
        self.c_tensor = torch.LongTensor([vocab[w] for w in self.choice.split()])
        self.d_pos_tensor = torch.LongTensor([pos_vocab[w] for w in self.d_pos])
        self.q_pos_tensor = torch.LongTensor([pos_vocab[w] for w in self.q_pos])
        self.d_ner_tensor = torch.LongTensor([ner_vocab[w] for w in self.d_ner])
        self.features = torch.from_numpy(self.features).type(torch.FloatTensor)
        self.p_q_relation = torch.LongTensor([rel_vocab[r] for r in input_dict['p_q_relation']])
        self.p_c_relation = torch.LongTensor([rel_vocab[r] for r in input_dict['p_c_relation']])

    def __str__(self):
        return 'Passage: %s\n Question: %s\n Answer: %s, Label: %d' % (self.passage, self.question, self.choice, self.label)

def _to_indices_and_mask(batch_tensor, need_mask=True):
    mx_len = max([t.size(0) for t in batch_tensor])
    batch_size = len(batch_tensor)
    indices = torch.LongTensor(batch_size, mx_len).fill_(0)
    if need_mask:
        mask = torch.ByteTensor(batch_size, mx_len).fill_(1)
    for i, t in enumerate(batch_tensor):
        indices[i, :len(t)].copy_(t)
        if need_mask:
            mask[i, :len(t)].fill_(0)
    if need_mask:
        return indices, mask
    else:
        return indices

def _to_feature_tensor(features):
    mx_len = max([f.size(0) for f in features])
    batch_size = len(features)
    f_dim = features[0].size(1)
    f_tensor = torch.FloatTensor(batch_size, mx_len, f_dim).fill_(0)
    for i, f in enumerate(features):
        f_tensor[i, :len(f), :].copy_(f)
    return f_tensor

def batchify(batch_data):
    p, p_mask = _to_indices_and_mask([ex.d_tensor for ex in batch_data])
    p_pos = _to_indices_and_mask([ex.d_pos_tensor for ex in batch_data], need_mask=False)
    p_ner = _to_indices_and_mask([ex.d_ner_tensor for ex in batch_data], need_mask=False)
    p_q_relation = _to_indices_and_mask([ex.p_q_relation for ex in batch_data], need_mask=False)
    p_c_relation = _to_indices_and_mask([ex.p_c_relation for ex in batch_data], need_mask=False)
    q, q_mask = _to_indices_and_mask([ex.q_tensor for ex in batch_data])
    q_pos = _to_indices_and_mask([ex.q_pos_tensor for ex in batch_data], need_mask=False)
    choices = [ex.choice.split() for ex in batch_data]
    c, c_mask = _to_indices_and_mask([ex.c_tensor for ex in batch_data])
    f_tensor = _to_feature_tensor([ex.features for ex in batch_data])
    y = torch.FloatTensor([ex.label for ex in batch_data])
    return p, p_pos, p_ner, p_mask, q, q_pos, q_mask, c, c_mask, f_tensor, p_q_relation, p_c_relation, y


In [19]:
"""Definitions of model layers/NN modules"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class StackedBRNN(nn.Module):
    """Stacked Bi-directional RNNs.

    Differs from standard PyTorch library in that it has the option to save
    and concat the hidden states between layers. (i.e. the output hidden size
    for each sequence input is num_layers * hidden_size).
    """

    def __init__(self, input_size, hidden_size, num_layers,
                 dropout_rate=0, dropout_output=False, rnn_type=nn.LSTM,
                 concat_layers=False, padding=False):
        super(StackedBRNN, self).__init__()
        self.padding = padding
        self.dropout_output = dropout_output
        self.dropout_rate = dropout_rate
        self.num_layers = num_layers
        self.concat_layers = concat_layers
        self.rnns = nn.ModuleList()
        for i in range(num_layers):
            input_size = input_size if i == 0 else 2 * hidden_size
            self.rnns.append(rnn_type(input_size, hidden_size,
                                      num_layers=1,
                                      bidirectional=True))

    def forward(self, x, x_mask):
        """Encode either padded or non-padded sequences.

        Can choose to either handle or ignore variable length sequences.
        Always handle padding in eval.

        Args:
            x: batch * len * hdim
            x_mask: batch * len (1 for padding, 0 for true)
        Output:
            x_encoded: batch * len * hdim_encoded
        """
        if x_mask.data.sum() == 0:
            # No padding necessary.
            output = self._forward_unpadded(x, x_mask)
        elif self.padding or not self.training:
            # Pad if we care or if its during eval.
            output = self._forward_padded(x, x_mask)
        else:
            # We don't care.
            output = self._forward_unpadded(x, x_mask)

        return output.contiguous()

    def _forward_unpadded(self, x, x_mask):
        """Faster encoding that ignores any padding."""
        # Transpose batch and sequence dims
        x = x.transpose(0, 1)

        # Encode all layers
        outputs = [x]
        for i in range(self.num_layers):
            rnn_input = outputs[-1]

            # Apply dropout to hidden input
            if self.dropout_rate > 0:
                rnn_input = F.dropout(rnn_input,
                                      p=self.dropout_rate,
                                      training=self.training)
            # Forward
            rnn_output = self.rnns[i](rnn_input)[0]
            outputs.append(rnn_output)

        # Concat hidden layers
        if self.concat_layers:
            output = torch.cat(outputs[1:], 2)
        else:
            output = outputs[-1]

        # Transpose back
        output = output.transpose(0, 1)

        # Dropout on output layer
        if self.dropout_output and self.dropout_rate > 0:
            output = F.dropout(output,
                               p=self.dropout_rate,
                               training=self.training)
        return output

    def _forward_padded(self, x, x_mask):
        """Slower (significantly), but more precise, encoding that handles
        padding.
        """
        # Compute sorted sequence lengths
        lengths = x_mask.data.eq(0).long().sum(1).squeeze()
        _, idx_sort = torch.sort(lengths, dim=0, descending=True)
        _, idx_unsort = torch.sort(idx_sort, dim=0)

        lengths = list(lengths[idx_sort])
        idx_sort = Variable(idx_sort)
        idx_unsort = Variable(idx_unsort)

        # Sort x
        x = x.index_select(0, idx_sort)

        # Transpose batch and sequence dims
        x = x.transpose(0, 1)

        # Pack it up
        rnn_input = nn.utils.rnn.pack_padded_sequence(x, lengths)

        # Encode all layers
        outputs = [rnn_input]
        for i in range(self.num_layers):
            rnn_input = outputs[-1]

            # Apply dropout to input
            if self.dropout_rate > 0:
                dropout_input = F.dropout(rnn_input.data,
                                          p=self.dropout_rate,
                                          training=self.training)
                rnn_input = nn.utils.rnn.PackedSequence(dropout_input,
                                                        rnn_input.batch_sizes)
            outputs.append(self.rnns[i](rnn_input)[0])

        # Unpack everything
        for i, o in enumerate(outputs[1:], 1):
            outputs[i] = nn.utils.rnn.pad_packed_sequence(o)[0]

        # Concat hidden layers or take final
        if self.concat_layers:
            output = torch.cat(outputs[1:], 2)
        else:
            output = outputs[-1]

        # Transpose and unsort
        output = output.transpose(0, 1)
        output = output.index_select(0, idx_unsort)

        # Pad up to original batch sequence length
        if output.size(1) != x_mask.size(1):
            padding = torch.zeros(output.size(0),
                                  x_mask.size(1) - output.size(1),
                                  output.size(2)).type(output.data.type())
            output = torch.cat([output, Variable(padding)], 1)

        # Dropout on output layer
        if self.dropout_output and self.dropout_rate > 0:
            output = F.dropout(output,
                               p=self.dropout_rate,
                               training=self.training)
        return output


class SeqAttnMatch(nn.Module):
    """Given sequences X and Y, match sequence Y to each element in X.

    * o_i = sum(alpha_j * y_j) for i in X
    * alpha_j = softmax(y_j * x_i)
    """

    def __init__(self, input_size, identity=False):
        super(SeqAttnMatch, self).__init__()
        if not identity:
            self.linear = nn.Linear(input_size, input_size)
        else:
            self.linear = None

    def forward(self, x, y, y_mask):
        """
        Args:
            x: batch * len1 * hdim
            y: batch * len2 * hdim
            y_mask: batch * len2 (1 for padding, 0 for true)
        Output:
            matched_seq: batch * len1 * hdim
        """
        # Project vectors
        if self.linear:
            x_proj = self.linear(x.view(-1, x.size(2))).view(x.size())
            x_proj = F.relu(x_proj)
            y_proj = self.linear(y.view(-1, y.size(2))).view(y.size())
            y_proj = F.relu(y_proj)
        else:
            x_proj = x
            y_proj = y

        # Compute scores
        scores = x_proj.bmm(y_proj.transpose(2, 1))

        # Mask padding
        y_mask = y_mask.unsqueeze(1).expand(scores.size())
        scores.data.masked_fill_(y_mask.data, -float('inf'))

        # Normalize with softmax
        alpha_flat = F.softmax(scores.view(-1, y.size(1)))
        alpha = alpha_flat.view(-1, x.size(1), y.size(1))

        # Take weighted average
        matched_seq = alpha.bmm(y)
        return matched_seq


class BilinearSeqAttn(nn.Module):
    """A bilinear attention layer over a sequence X w.r.t y:

    * o_i = softmax(x_i'Wy) for x_i in X.

    Optionally don't normalize output weights.
    """

    def __init__(self, x_size, y_size, identity=False, normalize=True):
        super(BilinearSeqAttn, self).__init__()
        self.normalize = normalize

        # If identity is true, we just use a dot product without transformation.
        if not identity:
            self.linear = nn.Linear(y_size, x_size)
        else:
            self.linear = None

    def forward(self, x, y, x_mask):
        """
        Args:
            x: batch * len * hdim1
            y: batch * hdim2
            x_mask: batch * len (1 for padding, 0 for true)
        Output:
            alpha = batch * len
        """
        Wy = self.linear(y) if self.linear is not None else y
        xWy = x.bmm(Wy.unsqueeze(2)).squeeze(2)
        xWy.data.masked_fill_(x_mask.data, -float('inf'))
        if self.normalize:
            alpha = F.softmax(xWy)
        else:
            alpha = xWy.exp()
        return alpha


class LinearSeqAttn(nn.Module):
    """Self attention over a sequence:

    * o_i = softmax(Wx_i) for x_i in X.
    """

    def __init__(self, input_size):
        super(LinearSeqAttn, self).__init__()
        self.linear = nn.Linear(input_size, 1)

    def forward(self, x, x_mask):
        """
        Args:
            x: batch * len * hdim
            x_mask: batch * len (1 for padding, 0 for true)
        Output:
            alpha: batch * len
        """
        x_flat = x.view(-1, x.size(-1))
        scores = self.linear(x_flat).view(x.size(0), x.size(1))
        scores.data.masked_fill_(x_mask.data, -float('inf'))
        alpha = F.softmax(scores)
        return alpha


# ------------------------------------------------------------------------------
# Functional
# ------------------------------------------------------------------------------


def uniform_weights(x, x_mask):
    """Return uniform weights over non-masked x (a sequence of vectors).

    Args:
        x: batch * len * hdim
        x_mask: batch * len (1 for padding, 0 for true)
    Output:
        x_avg: batch * hdim
    """
    alpha = Variable(torch.ones(x.size(0), x.size(1)))
    if x.data.is_cuda:
        alpha = alpha.cuda()
    alpha = alpha * x_mask.eq(0).float()
    alpha = alpha / alpha.sum(1).expand(alpha.size())
    return alpha


def weighted_avg(x, weights):
    """Return a weighted average of x (a sequence of vectors).

    Args:
        x: batch * len * hdim
        weights: batch * len, sum(dim = 1) = 1
    Output:
        x_avg: batch * hdim
    """
    return weights.unsqueeze(1).bmm(x).squeeze(1)


In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import layers
from utils import vocab, pos_vocab, ner_vocab, rel_vocab

class TriAN(nn.Module):

    def __init__(self, args):
        super(TriAN, self).__init__()
        self.args = args
        self.embedding_dim = 300
        self.embedding = nn.Embedding(len(vocab), self.embedding_dim, padding_idx=0)
        self.embedding.weight.data.fill_(0)
        self.embedding.weight.data[:2].normal_(0, 0.1)
        self.pos_embedding = nn.Embedding(len(pos_vocab), args.pos_emb_dim, padding_idx=0)
        self.pos_embedding.weight.data.normal_(0, 0.1)
        self.ner_embedding = nn.Embedding(len(ner_vocab), args.ner_emb_dim, padding_idx=0)
        self.ner_embedding.weight.data.normal_(0, 0.1)
        self.rel_embedding = nn.Embedding(len(rel_vocab), args.rel_emb_dim, padding_idx=0)
        self.rel_embedding.weight.data.normal_(0, 0.1)
        self.RNN_TYPES = {'lstm': nn.LSTM, 'gru': nn.GRU}

        self.p_q_emb_match = layers.SeqAttnMatch(self.embedding_dim)
        self.c_q_emb_match = layers.SeqAttnMatch(self.embedding_dim)
        self.c_p_emb_match = layers.SeqAttnMatch(self.embedding_dim)

        # Input size to RNN: word emb + question emb + pos emb + ner emb + manual features
        doc_input_size = 2 * self.embedding_dim + args.pos_emb_dim + args.ner_emb_dim + 5 + 2 * args.rel_emb_dim

        # RNN document encoder
        self.doc_rnn = layers.StackedBRNN(
            input_size=doc_input_size,
            hidden_size=args.hidden_size,
            num_layers=args.doc_layers,
            dropout_rate=0,
            dropout_output=args.dropout_rnn_output,
            concat_layers=False,
            rnn_type=self.RNN_TYPES[args.rnn_type],
            padding=args.rnn_padding)

        # RNN question encoder: word emb + pos emb
        qst_input_size = self.embedding_dim + args.pos_emb_dim
        self.question_rnn = layers.StackedBRNN(
            input_size=qst_input_size,
            hidden_size=args.hidden_size,
            num_layers=1,
            dropout_rate=0,
            dropout_output=args.dropout_rnn_output,
            concat_layers=False,
            rnn_type=self.RNN_TYPES[args.rnn_type],
            padding=args.rnn_padding)

        # RNN answer encoder
        choice_input_size = 3 * self.embedding_dim
        self.choice_rnn = layers.StackedBRNN(
            input_size=choice_input_size,
            hidden_size=args.hidden_size,
            num_layers=1,
            dropout_rate=0,
            dropout_output=args.dropout_rnn_output,
            concat_layers=False,
            rnn_type=self.RNN_TYPES[args.rnn_type],
            padding=args.rnn_padding)

        # Output sizes of rnn encoders
        doc_hidden_size = 2 * args.hidden_size
        question_hidden_size = 2 * args.hidden_size
        choice_hidden_size = 2 * args.hidden_size

        # Answer merging
        self.c_self_attn = layers.LinearSeqAttn(choice_hidden_size)
        self.q_self_attn = layers.LinearSeqAttn(question_hidden_size)

        self.p_q_attn = layers.BilinearSeqAttn(x_size=doc_hidden_size, y_size=question_hidden_size)

        self.p_c_bilinear = nn.Linear(doc_hidden_size, choice_hidden_size)
        self.q_c_bilinear = nn.Linear(question_hidden_size, choice_hidden_size)

    def forward(self, p, p_pos, p_ner, p_mask, q, q_pos, q_mask, c, c_mask, f_tensor, p_q_relation, p_c_relation):
        p_emb, q_emb, c_emb = self.embedding(p), self.embedding(q), self.embedding(c)
        p_pos_emb, p_ner_emb, q_pos_emb = self.pos_embedding(p_pos), self.ner_embedding(p_ner), self.pos_embedding(q_pos)
        p_q_rel_emb, p_c_rel_emb = self.rel_embedding(p_q_relation), self.rel_embedding(p_c_relation)

        # Dropout on embeddings
        if self.args.dropout_emb > 0:
            p_emb = nn.functional.dropout(p_emb, p=self.args.dropout_emb, training=self.training)
            q_emb = nn.functional.dropout(q_emb, p=self.args.dropout_emb, training=self.training)
            c_emb = nn.functional.dropout(c_emb, p=self.args.dropout_emb, training=self.training)
            p_pos_emb = nn.functional.dropout(p_pos_emb, p=self.args.dropout_emb, training=self.training)
            p_ner_emb = nn.functional.dropout(p_ner_emb, p=self.args.dropout_emb, training=self.training)
            q_pos_emb = nn.functional.dropout(q_pos_emb, p=self.args.dropout_emb, training=self.training)
            p_q_rel_emb = nn.functional.dropout(p_q_rel_emb, p=self.args.dropout_emb, training=self.training)
            p_c_rel_emb = nn.functional.dropout(p_c_rel_emb, p=self.args.dropout_emb, training=self.training)

        p_q_weighted_emb = self.p_q_emb_match(p_emb, q_emb, q_mask)
        c_q_weighted_emb = self.c_q_emb_match(c_emb, q_emb, q_mask)
        c_p_weighted_emb = self.c_p_emb_match(c_emb, p_emb, p_mask)
        p_q_weighted_emb = nn.functional.dropout(p_q_weighted_emb, p=self.args.dropout_emb, training=self.training)
        c_q_weighted_emb = nn.functional.dropout(c_q_weighted_emb, p=self.args.dropout_emb, training=self.training)
        c_p_weighted_emb = nn.functional.dropout(c_p_weighted_emb, p=self.args.dropout_emb, training=self.training)
        # print('p_q_weighted_emb', p_q_weighted_emb.size())

        p_rnn_input = torch.cat([p_emb, p_q_weighted_emb, p_pos_emb, p_ner_emb, f_tensor, p_q_rel_emb, p_c_rel_emb], dim=2)
        c_rnn_input = torch.cat([c_emb, c_q_weighted_emb, c_p_weighted_emb], dim=2)
        q_rnn_input = torch.cat([q_emb, q_pos_emb], dim=2)
        # print('p_rnn_input', p_rnn_input.size())

        p_hiddens = self.doc_rnn(p_rnn_input, p_mask)
        c_hiddens = self.choice_rnn(c_rnn_input, c_mask)
        q_hiddens = self.question_rnn(q_rnn_input, q_mask)
        # print('p_hiddens', p_hiddens.size())

        q_merge_weights = self.q_self_attn(q_hiddens, q_mask)
        q_hidden = layers.weighted_avg(q_hiddens, q_merge_weights)

        p_merge_weights = self.p_q_attn(p_hiddens, q_hidden, p_mask)
        # [batch_size, 2*hidden_size]
        p_hidden = layers.weighted_avg(p_hiddens, p_merge_weights)
        # print('p_hidden', p_hidden.size())

        c_merge_weights = self.c_self_attn(c_hiddens, c_mask)
        # [batch_size, 2*hidden_size]
        c_hidden = layers.weighted_avg(c_hiddens, c_merge_weights)
        # print('c_hidden', c_hidden.size())

        logits = torch.sum(self.p_c_bilinear(p_hidden) * c_hidden, dim=-1)
        logits += torch.sum(self.q_c_bilinear(q_hidden) * c_hidden, dim=-1)
        proba = F.sigmoid(logits)
        # print('proba', proba.size())

        return proba


In [24]:
import logging
import copy
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim.lr_scheduler as lr_scheduler
import numpy as np

from utils import vocab
from doc import batchify
from trian import TriAN

logger = logging.getLogger()

class Model:

    def __init__(self, args):
        self.args = args
        self.batch_size = args.batch_size
        self.finetune_topk = args.finetune_topk
        self.lr = args.lr
        self.use_cuda = (args.use_cuda == True) and torch.cuda.is_available()
        print('Use cuda:', self.use_cuda)
        if self.use_cuda:
            torch.cuda.set_device(int(args.gpu))
        self.network = TriAN(args)
        self.init_optimizer()
        if args.pretrained:
            print('Load pretrained model from %s...' % args.pretrained)
            self.load(args.pretrained)
        else:
            self.load_embeddings(vocab.tokens(), args.embedding_file)
        self.network.register_buffer('fixed_embedding', self.network.embedding.weight.data[self.finetune_topk:].clone())
        if self.use_cuda:
            self.network.cuda()
        print(self.network)
        self._report_num_trainable_parameters()

    def _report_num_trainable_parameters(self):
        num_parameters = 0
        for p in self.network.parameters():
            if p.requires_grad:
                sz = list(p.size())
                if sz[0] == len(vocab):
                    sz[0] = self.finetune_topk
                num_parameters += np.prod(sz)
        print('Number of parameters: ', num_parameters)

    def train(self, train_data):
        self.network.train()
        self.updates = 0
        iter_cnt, num_iter = 0, (len(train_data) + self.batch_size - 1) // self.batch_size
        for batch_input in self._iter_data(train_data):
            feed_input = [x for x in batch_input[:-1]]
            y = batch_input[-1]
            pred_proba = self.network(*feed_input)

            loss = F.binary_cross_entropy(pred_proba, y)
            self.optimizer.zero_grad()
            loss.backward()

            torch.nn.utils.clip_grad_norm(self.network.parameters(), self.args.grad_clipping)

            # Update parameters
            self.optimizer.step()
            self.network.embedding.weight.data[self.finetune_topk:] = self.network.fixed_embedding
            self.updates += 1
            iter_cnt += 1

            if self.updates % 20 == 0:
                print('Iter: %d/%d, Loss: %f' % (iter_cnt, num_iter, loss.data[0]))
        self.scheduler.step()
        print('LR:', self.scheduler.get_lr()[0])

    def evaluate(self, dev_data, debug=False, eval_train=False):
        if len(dev_data) == 0:
            return -1.0
        self.network.eval()
        correct, total, prediction, gold = 0, 0, [], []
        dev_data = sorted(dev_data, key=lambda ex: ex.id)
        for batch_input in self._iter_data(dev_data):
            feed_input = [x for x in batch_input[:-1]]
            y = batch_input[-1].data.cpu().numpy()
            pred_proba = self.network(*feed_input)
            pred_proba = pred_proba.data.cpu()
            prediction += list(pred_proba)
            gold += [int(label) for label in y]
            assert(len(prediction) == len(gold))

        if eval_train:
            prediction = [1 if p > 0.5 else 0 for p in prediction]
            acc = sum([1 if y1 == y2 else 0 for y1, y2 in zip(prediction, gold)]) / len(gold)
            return acc

        cur_pred, cur_gold, cur_choices = [], [], []
        if debug:
            writer = open('./data/output.log', 'w', encoding='utf-8')
        for i, ex in enumerate(dev_data):
            if i + 1 == len(dev_data):
                cur_pred.append(prediction[i])
                cur_gold.append(gold[i])
                cur_choices.append(ex.choice)
            if (i > 0 and ex.id[:-1] != dev_data[i - 1].id[:-1]) or (i + 1 == len(dev_data)):
                py, gy = np.argmax(cur_pred), np.argmax(cur_gold)
                if debug:
                    writer.write('Passage: %s\n' % dev_data[i - 1].passage)
                    writer.write('Question: %s\n' % dev_data[i - 1].question)
                    for idx, choice in enumerate(cur_choices):
                        writer.write('*' if idx == gy else ' ')
                        writer.write('%s  %f\n' % (choice, cur_pred[idx]))
                    writer.write('\n')
                if py == gy:
                    correct += 1
                total += 1
                cur_pred, cur_gold, cur_choices = [], [], []
            cur_pred.append(prediction[i])
            cur_gold.append(gold[i])
            cur_choices.append(ex.choice)

        acc = 1.0 * correct / total
        if debug:
            writer.write('Accuracy: %f\n' % acc)
            writer.close()
        return acc

    def predict(self, test_data):
        # DO NOT SHUFFLE test_data
        self.network.eval()
        prediction = []
        for batch_input in self._iter_data(test_data):
            feed_input = [x for x in batch_input[:-1]]
            pred_proba = self.network(*feed_input)
            pred_proba = pred_proba.data.cpu()
            prediction += list(pred_proba)
        return prediction

    def _iter_data(self, data):
        num_iter = (len(data) + self.batch_size - 1) // self.batch_size
        for i in range(num_iter):
            start_idx = i * self.batch_size
            batch_data = data[start_idx:(start_idx + self.batch_size)]
            batch_input = batchify(batch_data)

            # Transfer to GPU
            if self.use_cuda:
                batch_input = [Variable(x.cuda(async=True)) for x in batch_input]
            else:
                batch_input = [Variable(x) for x in batch_input]
            yield batch_input

    def load_embeddings(self, words, embedding_file):
        """Load pretrained embeddings for a given list of words, if they exist.

        Args:
            words: iterable of tokens. Only those that are indexed in the
              dictionary are kept.
            embedding_file: path to text file of embeddings, space separated.
        """
        words = {w for w in words if w in vocab}
        logger.info('Loading pre-trained embeddings for %d words from %s' %
                    (len(words), embedding_file))
        embedding = self.network.embedding.weight.data

        # When normalized, some words are duplicated. (Average the embeddings).
        vec_counts = {}
        with open(embedding_file) as f:
            for line in f:
                parsed = line.rstrip().split(' ')
                assert(len(parsed) == embedding.size(1) + 1)
                w = vocab.normalize(parsed[0])
                if w in words:
                    vec = torch.Tensor([float(i) for i in parsed[1:]])
                    if w not in vec_counts:
                        vec_counts[w] = 1
                        embedding[vocab[w]].copy_(vec)
                    else:
                        logging.warning('WARN: Duplicate embedding found for %s' % w)
                        vec_counts[w] = vec_counts[w] + 1
                        embedding[vocab[w]].add_(vec)

        for w, c in vec_counts.items():
            embedding[vocab[w]].div_(c)

        logger.info('Loaded %d embeddings (%.2f%%)' %
                    (len(vec_counts), 100 * len(vec_counts) / len(words)))

    def init_optimizer(self):
        parameters = [p for p in self.network.parameters() if p.requires_grad]
        if self.args.optimizer == 'sgd':
            self.optimizer = optim.SGD(parameters, self.lr,
                                       momentum=0.4,
                                       weight_decay=0)
        elif self.args.optimizer == 'adamax':
            self.optimizer = optim.Adamax(parameters,
                                        lr=self.lr,
                                        weight_decay=0)
        else:
            raise RuntimeError('Unsupported optimizer: %s' %
                               self.args.optimizer)
        self.scheduler = lr_scheduler.MultiStepLR(self.optimizer, milestones=[10, 15], gamma=0.5)

    def save(self, ckt_path):
        state_dict = copy.copy(self.network.state_dict())
        if 'fixed_embedding' in state_dict:
            state_dict.pop('fixed_embedding')
        params = {'state_dict': state_dict}
        torch.save(params, ckt_path)

    def load(self, ckt_path):
        logger.info('Loading model %s' % ckt_path)
        saved_params = torch.load(ckt_path, map_location=lambda storage, loc: storage)
        state_dict = saved_params['state_dict']
        return self.network.load_state_dict(state_dict)

    def cuda(self):
        self.use_cuda = True
        self.network.cuda()

In [ ]:
import os
import time
import torch
import random
import numpy as np

from datetime import datetime

from utils import load_data, build_vocab
from config import args
from model import Model

torch.manual_seed(args.seed)
np.random.seed(args.seed)
random.seed(args.seed)

if __name__ == '__main__':

    build_vocab()
    train_data = load_data('./data/train-data-processed.json')
    train_data += load_data('./data/trial-data-processed.json')
    dev_data = load_data('./data/dev-data-processed.json')
    if args.test_mode:
        # use validation data as training data
        train_data += dev_data
        dev_data = []
    model = Model(args)

    best_dev_acc = 0.0
    os.makedirs('./checkpoint', exist_ok=True)
    checkpoint_path = './checkpoint/%d-%s.mdl' % (args.seed, datetime.now().isoformat())
    print('Trained model will be saved to %s' % checkpoint_path)

    for i in range(args.epoch):
        print('Epoch %d...' % i)
        if i == 0:
            dev_acc = model.evaluate(dev_data)
            print('Dev accuracy: %f' % dev_acc)
        start_time = time.time()
        np.random.shuffle(train_data)
        cur_train_data = train_data

        model.train(cur_train_data)
        train_acc = model.evaluate(train_data[:2000], debug=False, eval_train=True)
        print('Train accuracy: %f' % train_acc)
        dev_acc = model.evaluate(dev_data, debug=True)
        print('Dev accuracy: %f' % dev_acc)

        if dev_acc > best_dev_acc:
            best_dev_acc = dev_acc
            os.system('mv ./data/output.log ./data/best-dev.log')
            model.save(checkpoint_path)
        elif args.test_mode:
            model.save(checkpoint_path)
        print('Epoch %d use %d seconds.' % (i, time.time() - start_time))

    print('Best dev accuracy: %f' % best_dev_acc)


Load vocabulary from ./data/vocab...
Vocabulary size: 33382
Load pos vocabulary from ./data/pos_vocab...
POS vocabulary size: 51
Load ner vocabulary from ./data/ner_vocab...
NER vocabulary size: 20
Load relation vocabulary from ./data/rel_vocab...
Rel vocabulary size: 39
Load 19462 examples from ./data/train-data-processed.json...
Load 1020 examples from ./data/trial-data-processed.json...
Load 2822 examples from ./data/dev-data-processed.json...
Use cuda: False


TriAN(
  (embedding): Embedding(33382, 300, padding_idx=0)
  (pos_embedding): Embedding(51, 12, padding_idx=0)
  (ner_embedding): Embedding(20, 8, padding_idx=0)
  (rel_embedding): Embedding(39, 10, padding_idx=0)
  (p_q_emb_match): SeqAttnMatch(
    (linear): Linear(in_features=300, out_features=300, bias=True)
  )
  (c_q_emb_match): SeqAttnMatch(
    (linear): Linear(in_features=300, out_features=300, bias=True)
  )
  (c_p_emb_match): SeqAttnMatch(
    (linear): Linear(in_features=300, out_features=300, bias=True)
  )
  (doc_rnn): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(645, 96, bidirectional=True)
    )
  )
  (question_rnn): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(312, 96, bidirectional=True)
    )
  )
  (choice_rnn): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(900, 96, bidirectional=True)
    )
  )
  (c_self_attn): LinearSeqAttn(
    (linear): Linear(in_features=192, out_features=1, bias=True)
  )
  (q_self_attn): LinearSeqAttn(
    (linear): Line

/media/niranjan/Data Science/INKERS/Project -1/Project-Final/layers.py:197: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  alpha_flat = F.softmax(scores.view(-1, y.size(1)))
/media/niranjan/Data Science/INKERS/Project -1/Project-Final/layers.py:263: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  alpha = F.softmax(scores)
/media/niranjan/Data Science/INKERS/Project -1/Project-Final/layers.py:236: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  alpha = F.softmax(xWy)


Dev accuracy: 0.508859


/media/niranjan/Data Science/INKERS/Project -1/Project-Final/model.py:63: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(self.network.parameters(), self.args.grad_clipping)
/media/niranjan/Data Science/INKERS/Project -1/Project-Final/model.py:72: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  print('Iter: %d/%d, Loss: %f' % (iter_cnt, num_iter, loss.data[0]))


Iter: 20/641, Loss: 0.698544
Iter: 40/641, Loss: 0.717111
Iter: 60/641, Loss: 0.638708
Iter: 80/641, Loss: 0.670520


In [ ]:
#  Play with Pre-Trained Model.. 
from config import args
from utils import load_data, build_vocab, gen_submission, gen_final_submission, eval_based_on_outputs
from model import Model

if __name__ == '__main__':
    if not args.pretrained:
        print('No pretrained model specified.')
        exit(0)
    build_vocab()

    if args.test_mode:
        dev_data = load_data('./data/test-data-processed.json')
    else:
        dev_data = load_data('./data/dev-data-processed.json')
    model_path_list = args.pretrained.split(',')
    for model_path in model_path_list:
        print('Load model from %s...' % model_path)
        args.pretrained = model_path
        model = Model(args)

        # evaluate on development dataset
        dev_acc = model.evaluate(dev_data)
        print('dev accuracy: %f' % dev_acc)

        # generate submission zip file for Codalab
        prediction = model.predict(dev_data)
        gen_submission(dev_data, prediction)

    gen_final_submission(dev_data)
    if not args.test_mode:
        eval_based_on_outputs('./answer.txt')


### How to RUN
you Train model by simply Running python3 src/main.py --gpu 0 in commandline

To Run in Jupyter Notebook, a Details Code for Producing the Results are Below: